In [1]:
import pandas as pd
import numpy as np
import random
import networkx as nx
from tqdm import tqdm
import re
import matplotlib.pyplot as plt
import itertools
from operator import itemgetter
import pickle

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

In [2]:
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import optimizers, utils, initializers, regularizers
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
import linkpred
from linkpred.evaluation import Pair
import seaborn as sns

In [4]:
import sklearn.metrics as metrics

In [5]:
from gensim.models import Word2Vec

In [6]:
with open('universe_597.pickle', 'rb') as f:
    universe_597 = pickle.load(f)

In [7]:
len(universe_597)

43365

In [8]:
embedding_pickle_597 = pd.read_pickle('embedding_pickle_597.csv')

In [9]:
edges_597 = pd.read_pickle('edges_original_597.csv')

In [10]:
H_597 = nx.read_graphml("H_597.graphml")

In [11]:
all_unconnected_pairs_597 = []

In [12]:
def check_isequal(x, a, b):
    if x[0] == a:
        if x[1] == b:
            return True
    if x[1] == a:
        if x[0] == b:
            return True
    return False

In [13]:
for ui in universe_597:
    is_not = False
    for i,j in edges_597.values:
        if check_isequal(ui, i, j):
            is_not = True
            break
    if not is_not:
        all_unconnected_pairs_597.append((ui[0], ui[1]))

KeyboardInterrupt: 

In [ ]:
len(all_unconnected_pairs_597)

In [ ]:
word2vec_597 = Word2Vec.load("word2vec_597.model")

Collegamenti inesistenti => target negativo

In [ ]:
node_1_unlinked = [word2vec_597.wv[i[0]] for i in all_unconnected_pairs_597]
node_2_unlinked = [word2vec_597.wv[i[1]] for i in all_unconnected_pairs_597]

data_597 = pd.DataFrame({'node_1':node_1_unlinked, 
                     'node_2':node_2_unlinked})

# add target variable 'link'
data_597['link'] = 0

In [ ]:
data_597

In [ ]:
def omissibile_links(G, df_edges):
    initial_node_count = len(G.nodes)

    fb_df_temp = df_edges.copy()

    # empty list to store removable links
    omissible_links_index = []

    for i in tqdm(df_edges.index.values):

      # remove a node pair and build a new graph
      G_temp = nx.from_pandas_edgelist(fb_df_temp.drop(index = i), "first", "second", create_using=nx.Graph())

      # check there is no spliting of graph and number of nodes is same
      if (nx.number_connected_components(G_temp) == 1) and (len(G_temp.nodes) == initial_node_count):
        omissible_links_index.append(i)
        fb_df_temp = fb_df_temp.drop(index = i)
    return omissible_links_index

In [ ]:
omissible_links_597 =  omissibile_links(H_597, edges_597)

In [ ]:
len(edges_597)

In [ ]:
len(omissible_links_597)

In [ ]:
len(universe_597)

In [ ]:
# create dataframe of removable edges
fb_df_ghost_597 = embedding_pickle_597.loc[omissible_links_597]

fb_df_ghost_597 = fb_df_ghost_597.rename(columns={"first": "node_1", "second": "node_2"})
# add the target variable 'link'
fb_df_ghost_597['link'] = 1

data_597 = data_597.append(fb_df_ghost_597[['node_1', 'node_2', 'link']], ignore_index=True)

In [ ]:
data_597['link'].value_counts()

In [ ]:
data_597.to_pickle("data_597.csv")

----------------- Lettura dati -------------------

In [ ]:
data_597 = pd.read_pickle('data_597.csv')

# 2 - Classificatori

In [ ]:
X_597 = []
for i in range(data_597.shape[0]):
    first = data_597.iloc[i][0]
    second = data_597.iloc[i][1]
    mixed = np.concatenate((first, second), 0)
    X_597.append(mixed)

In [ ]:
xtrain_597, xtest_597, ytrain_597, ytest_597 = train_test_split(np.asarray(X_597), data_597['link'], 
                                                test_size = 0.3, 
                                                random_state = 35,
                                                stratify=data_597['link'])

### 1 - DecisionTree

In [ ]:
from sklearn import tree
from sklearn.model_selection import ShuffleSplit, StratifiedKFold, GridSearchCV
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [ ]:
# Set the parameters by cross-validation
tuned_parameters = {
    'criterion' : ['gini', 'entropy'],
    'max_depth': [2, 5, 10, 15, None],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 5, 10, 20],
}
scores = ['roc_auc']

In [ ]:
def builtGridSearch( X_train, X_test, y_train, y_test, tuned_parameters, scores):
    optimals = {}
    for score in scores:
        print("------- Score = " + str(score) + " ------- \n")
        model = tree.DecisionTreeClassifier()
        k_fold = StratifiedKFold(n_splits=5, random_state=42)
        
        print("> Fold = " + str(k_fold) + "\n")
        clf = GridSearchCV(model, tuned_parameters, error_score='raise', cv=5, scoring = score, return_train_score=True)

        clf.fit(X_train, y_train)

        print("> Best Parameter set: \n")
        best = clf.best_params_
        print(best)
        
        print("\n> Grid scores:\n")
        means = clf.cv_results_['mean_test_score']
        stds = clf.cv_results_['std_test_score']
        
        parameters = {"criterion": [], "max_depth": [], "min_samples_leaf": [], "min_samples_split": []}
        for mean, std, params in zip(means, stds, clf.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"
                  % (mean, std * 2, params))
            parameters["criterion"].append(params["criterion"])

            if(params["max_depth"] == None):
                parameters["max_depth"].append(0)
            else:
                parameters["max_depth"].append(params["max_depth"])

            #parameters["max_depth"].append(params["max_depth"])
            parameters["min_samples_leaf"].append(params["min_samples_leaf"])
            parameters["min_samples_split"].append(params["min_samples_split"])
        
        print("-> Report\n") 
        y_true, y_pred = y_test, clf.predict(X_test)
        print(classification_report(y_true, y_pred))
        print("\n")
        print("**** Matrice di Confusione *****")
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        print(' True negative: %d False negative: %d' % (tn, fn))
        print(' True positive: %d False positive: %d' % (tp, fp))

        
        # Creazione del csv
        d = {score: means, 'std_dev': stds*2, 'criterion': parameters["criterion"], "max_depth": parameters["max_depth"], "min_samples_leaf": parameters["min_samples_leaf"], "min_samples_split": parameters["min_samples_split"]}
        
        dataF = pd.DataFrame(data=d)
        
        dataF.to_csv("result_DT_gridsearch.csv",index=False, header=True)
        

        print("...........RESULTS FOR TRAINING.........")
        print("........................................")
        means = clf.cv_results_['mean_train_score']
        stds = clf.cv_results_['std_train_score']
        for mean, std, params in zip(means, stds, clf.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"
                  % (mean, std * 2, params))        
        
        print("____________________________________________")
        
        optimals[score] = best
    return optimals

In [ ]:
optimals_597 = builtGridSearch(xtrain_597, xtest_597, ytrain_597, ytest_597, tuned_parameters, scores)

In [ ]:
optimals_597

In [ ]:
model_597 = tree.DecisionTreeClassifier(criterion = "entropy", max_depth = None, min_samples_leaf = 20, min_samples_split = 10)
model_597.fit(xtrain_597, ytrain_597)
y_pred_597 = model_597.predict(xtest_597)

In [ ]:
y_score_597 = model_597.predict_proba(xtest_597)
fpr0, tpr0, _ = roc_curve(ytest_597, y_score_597[:, 1])
roc_auc0_597 = auc(fpr0, tpr0)

In [ ]:
plt.plot(fpr0, tpr0, color='darkorange', lw=3, label='$AUC_0$ = %.3f' % (roc_auc0_597))

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16)
plt.title('ROC curve', fontsize=16)
plt.legend(loc="lower right", fontsize=14, frameon=False)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.show()

### 1 - DecisionTree_Bilanciato

In [ ]:
remove_n = data_597['link'].value_counts()[0] - data_597['link'].value_counts()[1]
remove_n

In [ ]:
df = data_597.copy()
drop_indices = np.random.choice(df[df["link"]==0].index, remove_n, replace=False)
df_subset = df.drop(drop_indices)
print(df_subset['link'].value_counts())

X_sbset_597 = []
for i in range(df_subset.shape[0]):
    first = df_subset.iloc[i][0]
    second = df_subset.iloc[i][1]
    mixed = np.concatenate((first, second), 0)
    X_sbset_597.append(mixed)
    
    
xtrain_sb_597, xtest_sb_597, ytrain_sb_597, ytest_sb_597 = train_test_split(np.asarray(X_sbset_597), df_subset["link"], 
                                                test_size = 0.3, 
                                                random_state = 35,
                                                stratify=df_subset["link"])

In [ ]:
optimals_sb_597 = builtGridSearch(xtrain_sb_597, xtest_sb_597, ytrain_sb_597, ytest_sb_597, tuned_parameters, scores)

In [ ]:
model_sb_597 = tree.DecisionTreeClassifier(criterion = "gini", max_depth = None, min_samples_leaf = 20, min_samples_split = 10)
model_sb_597.fit(xtrain_sb_597, ytrain_sb_597)

In [ ]:
y_score_sb_597 = model_sb_597.predict_proba(xtest_sb_597)
fpr0, tpr0, _ = roc_curve(ytest_sb_597, y_score_sb_597[:, 1])
roc_auc0_dt_597_sb = auc(fpr0, tpr0)

In [ ]:
plt.plot(fpr0, tpr0, color='darkorange', lw=3, label='$AUC_0$ = %.3f' % (roc_auc0_dt_597_sb))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16)
plt.title('ROC curve', fontsize=16)
plt.legend(loc="lower right", fontsize=14, frameon=False)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.show()

### 2 - Regressione logistica

In [ ]:
lr = LogisticRegression()
lr.fit(xtrain_597, ytrain_597)
predictions = lr.predict_proba(xtest_597)

In [ ]:
y_score = lr.predict_proba(xtest_597)
fpr0, tpr0, _ = roc_curve(ytest_597, y_score[:, 1])
roc_auc0 = auc(fpr0, tpr0)
print(roc_auc0)

In [ ]:
plt.plot(fpr0, tpr0, color='darkorange', lw=3, label='$AUC_0$ = %.3f' % (roc_auc0))

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16)
plt.title('ROC curve', fontsize=16)
plt.legend(loc="lower right", fontsize=14, frameon=False)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.show()

### 2 - Regressione Logistica_Bilanciata

In [ ]:
lr = LogisticRegression()
lr.fit(xtrain_sb_597, ytrain_sb_597)

In [ ]:
y_score = lr.predict_proba(xtest_sb_597)
fpr0, tpr0, _ = roc_curve(ytest_sb_597, y_score[:, 1])
roc_auc0 = auc(fpr0, tpr0)
print(roc_auc0)

In [ ]:
plt.plot(fpr0, tpr0, color='darkorange', lw=3, label='$AUC_0$ = %.3f' % (roc_auc0))

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16)
plt.title('ROC curve', fontsize=16)
plt.legend(loc="lower right", fontsize=14, frameon=False)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.show()

### 3 - LGBM

In [ ]:
new_xtrain_597, xval_597, new_ytrain_597, yval_597 = train_test_split(xtrain_597, ytrain_597, 
                                                test_size = 0.3, 
                                                random_state = 35,
                                                stratify=ytrain_597)

In [ ]:
import lightgbm as lgbm

In [ ]:
train_data = lgbm.Dataset(new_xtrain_597, new_ytrain_597)
test_data = lgbm.Dataset(xval_597, yval_597)

# define parameters
parameters = {
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'num_threads' : 2,
    'seed' : 76
}

# train lightGBM model
model = lgbm.train(parameters,
                   train_data,
                   valid_sets=test_data,
                   num_boost_round=1000,
                   early_stopping_rounds=20)

y_pred = model.predict(xtest_597)

In [ ]:
y_pred

In [ ]:
fpr0, tpr0, _ = roc_curve(ytest_597, y_pred)
roc_auc_lgbm = auc(fpr0, tpr0)
print(roc_auc_lgbm)

### 3 - LGBM_Bilanciato

In [ ]:
new_xtrain_sb_597, xval_sb_597, new_ytrain_sb_597, yval_sb_597 = train_test_split(xtrain_sb_597, ytrain_sb_597, 
                                                test_size = 0.3, 
                                                random_state = 35,
                                                stratify=ytrain_sb_597)

In [ ]:
train_data = lgbm.Dataset(new_xtrain_sb_597, new_ytrain_sb_597)
test_data = lgbm.Dataset(xval_sb_597, yval_sb_597)

# define parameters
parameters = {
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'false',
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'num_threads' : 2,
    'seed' : 76
}

# train lightGBM model
model = lgbm.train(parameters,
                   train_data,
                   valid_sets=test_data,
                   num_boost_round=1000,
                   early_stopping_rounds=20)
y_pred = model.predict(xtest_sb_597)

In [ ]:
fpr0, tpr0, _ = roc_curve(ytest_sb_597, y_pred)
roc_auc_lgbm_sb = auc(fpr0, tpr0)
print(roc_auc_lgbm_sb)

### 3 - LinearSVC

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV

In [ ]:
# Set the parameters by cross-validation
tuned_parameters = {
    'tol':(1.0, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6),
    'C': (0.001, 0.05, 0.01, 0.1, 1.0, 10.0, 50, 100.0)
}
# Set the parameters by cross-validation
k_fold = StratifiedKFold(n_splits=4, random_state=42)

scores = ['roc_auc']

def gridsearch_linearsvc(X_train, y_train, X_test, y_test, k_fold, tuned_parameters, scores):
    optimals = {}
    for score in scores:
        print("# Tuning hyper-parameters for ----> %s" % score)
        print()

        svm = LinearSVC()

        clf = GridSearchCV(svm, tuned_parameters, error_score='raise', cv=k_fold, scoring=score)

        clf.fit(X_train, y_train)

        print("Best parameters set found on development set:")
        print()
        print(clf.best_params_)
        print()
        print("Grid scores on development set:")
        print()
        means = clf.cv_results_['mean_test_score']
        stds = clf.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"
                  % (mean, std * 2, params))
        print()

        print("Detailed classification report:")
        print()
        print("The model is trained on the full development set.")
        print("The scores are computed on the full evaluation set.")
        print()
        y_true, y_pred = y_test, clf.predict(X_test)
        print(classification_report(y_true, y_pred))
        print()
        print("**** Matrice di Confusione *****")
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        print(' True negative: %d False negative: %d' % (tn, fn))
        print(' True positive: %d False positive: %d' % (tp, fp))
        print("____________________________________________")
        optimals[score] = clf.best_params_
    return optimals

In [ ]:
optimals_597 = gridsearch_linearsvc(xtrain_597, ytrain_597, xtest_597, ytest_597, k_fold, tuned_parameters, scores)

In [ ]:
optimals_597

In [ ]:
model_597 = LinearSVC(C = 100.0, tol =1.0)
model_597.fit(xtrain_597, ytrain_597)


y_score_597 = model_597.predict(xtest_597)

cclf = CalibratedClassifierCV(base_estimator=model_597, cv=5)
cclf.fit(xtrain_597, ytrain_597)
y_pred = cclf.predict(xtest_597)
y_score_597 = cclf.predict_proba(xtest_597)
fpr0, tpr0, _ = roc_curve(ytest_597, y_score_597[:, 1])
roc_auc0 = auc(fpr0, tpr0)
print(roc_auc0)

plt.plot(fpr0, tpr0, color='darkorange', lw=3, label='$AUC_0$ = %.3f' % (roc_auc0))

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16)
plt.title('ROC curve', fontsize=16)
plt.legend(loc="lower right", fontsize=14, frameon=False)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.show()

### 3 - LinearSVC_Bilanciato

In [ ]:
optimals_sb_597 = gridsearch_linearsvc(xtrain_sb_597, ytrain_sb_597, xtest_sb_597, ytest_sb_597, k_fold, tuned_parameters, scores)

In [ ]:
optimals_sb_597

In [ ]:
model_sb_597 = LinearSVC(C = 10.0, tol =0.1)
model_sb_597.fit(xtrain_sb_597, ytrain_sb_597)


y_score_sb_597 = model_sb_597.predict(xtest_sb_597)

cclf = CalibratedClassifierCV(base_estimator=model_sb_597, cv=5)
cclf.fit(xtrain_sb_597, ytrain_sb_597)
y_pred = cclf.predict(xtest_sb_597)
y_score_sb_597 = cclf.predict_proba(xtest_sb_597)
fpr0, tpr0, _ = roc_curve(ytest_sb_597, y_score_sb_597[:, 1])
roc_auc0 = auc(fpr0, tpr0)
print(roc_auc0)

plt.plot(fpr0, tpr0, color='darkorange', lw=3, label='$AUC_0$ = %.3f' % (roc_auc0))

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16)
plt.title('ROC curve', fontsize=16)
plt.legend(loc="lower right", fontsize=14, frameon=False)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.show()

# 4 - NN

In [ ]:
def build_model_1( X_train):
    optimizer='adagrad'
    #optimizer='adam'
    n_feature = X_train.shape[1]
    model = Sequential()
    model.add(Dense(32, input_dim=n_feature, activation='tanh'))
    model.add(Dense(32, activation='tanh'))
    model.add(Dense(32, activation='tanh'))
    model.add(Dense(32, activation='tanh'))
    model.add(Dense(32, activation='tanh'))
    model.add(Dense(32, activation='tanh'))
    model.add(Dense(32, activation='tanh'))
    model.add(Dense(32, activation='tanh'))
    # Linear
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=["AUC"])
    return model

In [ ]:
def build_model_2(X_train):
    n_feature = X_train.shape[1]
    model = Sequential()
    model.add(Dense(128, input_dim=n_feature, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    # Linear
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01)))
    sgd = optimizers.SGD(learning_rate=0.1, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=["AUC"])
    return model

In [ ]:
model1_597 = build_model_1(xtrain_597)
history1_597 = model1_597.fit(xtrain_597, ytrain_597, validation_data=(xtest_597, ytest_597), epochs=30, batch_size=10).history

model2_597 = build_model_2(xtrain_597)
history2_597 = model2_597.fit(xtrain_597, ytrain_597, validation_data=(xtest_597, ytest_597), epochs=30, batch_size=50).history

In [ ]:
plt.plot(history1_597['loss'], label='Loss 10 (batch_size)')
plt.plot(history2_597['loss'], label='Loss 50 (batch_size)')
plt.xlabel('Epochs')
plt.ylabel('Cross-Entropy')
plt.legend()
plt.show()

In [ ]:
plt.plot(history1_597['loss'], label='Train')
plt.plot(history1_597['val_loss'], label='Val')
plt.xlabel('Epochs')
plt.ylabel('Cross-Entropy')
plt.legend()
plt.show()

In [ ]:
plt.plot(history2_597['loss'], label='Train')
plt.plot(history2_597['val_loss'], label='Val')
plt.xlabel('Epochs')
plt.ylabel('Cross-Entropy')
plt.legend()
plt.show()

In [ ]:
test_loss_1_597, test_auc_1_597 = model1_597.evaluate(xtest_597, ytest_597)
test_loss_2_597, test_auc_2_597 = model2_597.evaluate(xtest_597, ytest_597)

print('Loss %f, AUC %f' % (test_loss_1_597, test_auc_1_597))
print('Loss %f, AUC %f' % (test_loss_2_597, test_auc_2_597))

In [ ]:
def create_model(X_train, learning_rate=0.1, momentum = 0.9, nesterov = True,
                 activation = 'sigmoid', regularizer = 0.01, units=3, hidden_layers=1):
    # create model
    model = Sequential()
    #initializer = initializers.RandomUniform(minval=0, maxval=0.5, seed=None)
    model.add(Dense(units, input_dim=X_train.shape[1], activation = activation, 
                         kernel_regularizer=regularizers.l2(regularizer)))  
    for i in range(1,hidden_layers):
          model.add(Dense(units, activation = activation, 
                         kernel_regularizer=regularizers.l2(regularizer)))  
    model.add(Dense(1, activation = 'sigmoid',  kernel_regularizer=regularizers.l2(regularizer)))
    sgd = optimizers.SGD(learning_rate=learning_rate, momentum=momentum, nesterov=nesterov)
    model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['AUC'])
    return model

In [ ]:
param_list = {'batch_size': [128],
              'epochs': [100],
             'learning_rate': [0.261],
             'momentum':[0.9],
             'nesterov':[False],
             'activation':['tanh'],
              'regularizer':[0.0001],
              'units': [10],
             'hidden_layers': [1]}

In [ ]:
class Model:
    def __init__(self, loss = 0, accuracy = 0, batch_size = 0, epochs = 0, learning_rate=0, momentum=0,nesterov=False,
                activation='sigmoid', regularizer=0, units=0):
        self.accuracy_CV_list = []
        self.accuracy = accuracy
        self.loss_CV_list = []
        self.loss = loss
        self.mee_list = []
        self.mee = 0
        self.std = 0
        self.batch_size = batch_size
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.nesterov = nesterov
        self.activation = activation
        self.regularizer = regularizer
        self.units = units
        self.noise = 0
    def toString(self):
        print("""{MEE: %f (+/- %0.2f), Number of Units: %d, Batch Size: %d, Epochs: %d, Learning Rate: %f, Momentum: %f,
              "Nesterov: %s, Activation: %s, Regularization: %f, Noise: %f}""" % 
              (self.mee, self.std, self.units, self.batch_size, self.epochs, self.learning_rate, self.momentum,
              self.nesterov, self.activation, self.regularizer, self.noise))

In [ ]:
def GridSearchNN(X_train, Y_train, parameters, cv = 3, rand=False):
    models = []
    allNames = list(parameters)
    combinations = itertools.product(*(parameters[Name] for Name in allNames))
    searchList = []
    if rand:
        searchList = random.sample(list(combinations), k=100)
    else:
        searchList = list(combinations)
    for i in searchList:
        print(searchList.index(i), i)
        batch_size = i[0]
        epochs=i[1]
        learning_rate=i[2]
        momentum=i[3]
        nesterov=i[4]
        activation=i[5]
        regularizer=i[6]
        units = i[7]
        temp = Model(batch_size=batch_size, epochs=epochs, learning_rate=learning_rate,
                     momentum=momentum, nesterov=nesterov, activation=activation, regularizer=regularizer, units=units)
        model = None # Clearing the NN.
        model = create_model(X_train, learning_rate=learning_rate, momentum=momentum, nesterov=nesterov, 
                                 activation=activation, regularizer=regularizer, units=units)
        r = model.fit(X_train, Y_train,validation_split = float(1.0/cv),
                      batch_size =batch_size, epochs=epochs, verbose=1)
        loss = r.history['val_loss'][-1]
        #accuracy = r.history['val_acc'][-1]
        temp.loss = loss
        #temp.accuracy = accuracy
        models.append(temp) 
          
    result = sorted(models, key=lambda x: x.loss, reverse=False)
    return result, r, model

In [ ]:
allModels_597, r_597, model_597 = GridSearchNN(xtrain_597, ytrain_597, param_list, cv=3, rand=False)

In [ ]:
pred_597 = model_597.predict_classes(xtest_597)

In [ ]:
y_pred_keras_597 = model_597.predict(xtest_597).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(ytest_597, y_pred_keras_597)
auc_keras_597 = auc(fpr_keras, tpr_keras)

In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras 597 (area = {:.3f})'.format(auc_keras_597))

plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

# 4 - NN_Bilanciato

In [ ]:
model1_sb_597 = build_model_1(xtrain_sb_597)
history1_sb_597 = model1_sb_597.fit(xtrain_sb_597, ytrain_sb_597, validation_data=(xtest_sb_597, ytest_sb_597), epochs=30, batch_size=10).history

model2_sb_597 = build_model_2(xtrain_sb_597)
history2_sb_597 = model2_sb_597.fit(xtrain_sb_597, ytrain_sb_597, validation_data=(xtest_sb_597, ytest_sb_597), epochs=30, batch_size=50).history

In [ ]:
test_loss_1_sb_597, test_auc_1_sb_597 = model1_sb_597.evaluate(xtest_sb_597, ytest_sb_597)
test_loss_2_sb_597, test_auc_2_sb_597 = model2_sb_597.evaluate(xtest_sb_597, ytest_sb_597)

print('Loss %f, AUC %f' % (test_loss_1_sb_597, test_auc_1_sb_597))
print('Loss %f, AUC %f' % (test_loss_2_sb_597, test_auc_2_sb_597))

In [ ]:
allModels_sb_597, r_sb_597, model_sb_597 = GridSearchNN(xtrain_sb_597, ytrain_sb_597, param_list, cv=3, rand=False)

In [ ]:
pred_sb_597 = model_sb_597.predict_classes(xtest_sb_597)

In [ ]:
y_pred_keras_sb_597 = model_sb_597.predict(xtest_sb_597).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(ytest_sb_597, y_pred_keras_sb_597)
auc_keras_sb_597 = auc(fpr_keras, tpr_keras)

In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras bilancato 597 (area = {:.3f})'.format(auc_keras_sb_597))

plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

# ----- Curve d'apprendimento

In [ ]:
from sklearn.model_selection import learning_curve
def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes.set_title(title)
    if ylim is not None:
        axes.set_ylim(*ylim)
    axes.set_xlabel("Examples")
    axes.set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes,
                       return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes.grid()
    axes.fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes.fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes.plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes.plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes.legend(loc="best")
    return plt

In [ ]:
def plot_learning_dt(X1, y1, X2, y2):
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 5))

    cv = ShuffleSplit(n_splits=10, test_size=0.3, random_state=0)
    estimator = tree.DecisionTreeClassifier(criterion = "entropy", max_depth = None, min_samples_leaf = 20, min_samples_split = 10)
    title = "Decision Tree G1"
    plot_learning_curve(estimator, title, X1, y1, axes=axes[0], ylim=(0.3, 1.01),
                        cv=cv, n_jobs=10)
    print("done")
    cv = ShuffleSplit(n_splits=10, test_size=0.3, random_state=0)
    estimator = tree.DecisionTreeClassifier(criterion = "gini", max_depth = None, min_samples_leaf = 20, min_samples_split = 10)
    title = "Decision Tree G1 Bilanciato"
    plot_learning_curve(estimator, title, X2, y2, axes=axes[1], ylim=(0.3, 1.01),
                        cv=cv, n_jobs=10)
    plt.show()

In [ ]:
plot_learning_dt(xtrain_597, ytrain_597, xtrain_sb_597, ytrain_sb_597)

In [ ]:
def plot_learning_regr(X1, y1, X2, y2):
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 5))

    cv = ShuffleSplit(n_splits=10, test_size=0.3, random_state=0)
    estimator = LogisticRegression()
    title = "Logistic Regression 597"
    plot_learning_curve(estimator, title, X1, y1, axes=axes[0], ylim=(0.3, 1.01),
                        cv=cv, n_jobs=10)

    cv = ShuffleSplit(n_splits=10, test_size=0.3, random_state=0)
    estimator = LogisticRegression()
    title = "Logistic Regression 597 Bilanciato"
    plot_learning_curve(estimator, title, X2, y2, axes=axes[1], ylim=(0.3, 1.01),
                        cv=cv, n_jobs=10)
    plt.show()

In [ ]:
plot_learning_regr(xtrain_597, ytrain_597, xtrain_sb_597, ytrain_sb_597)